In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

C:\Users\graykar\Documents\Annaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
Titanic_Data = pd.read_csv("C:\\Users\\graykar\\Downloads\\Pyhton\\Data Files\\titanic.csv")

In [3]:
Titanic_Data.shape

(891, 12)

In [4]:
Titanic_Data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
X_train, X_test, y_train, y_test = train_test_split( Titanic_Data , Titanic_Data['Survived'] , test_size=0.20, random_state=1)

In [6]:
X_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            144
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          547
Embarked         2
dtype: int64

In [7]:
#Drop Feature which has more nulls
v_null_drop = X_train.isnull().mean()[X_train.isnull().mean() > 0.70].index
X_train.drop(v_null_drop , inplace=True, axis=1)
v_null_drop

X_train.drop(['PassengerId'], axis=1, inplace=True)

In [8]:
X_train.shape

(712, 10)

In [9]:
X_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
301,1,3,"McCoy, Mr. Bernard",male,NaN,2,0,367226,23.2500,Q
309,1,1,"Francatelli, Miss. Laura Mabel",female,30.0,0,0,PC 17485,56.9292,C
516,1,2,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.5000,S
120,0,2,"Hickman, Mr. Stanley George",male,21.0,2,0,S.O.C. 14879,73.5000,S
570,1,2,"Harris, Mr. George",male,62.0,0,0,S.W./PP 752,10.5000,S


In [ ]:
X_train.Fare.hist()

In [10]:
Embarked_mode_Value = X_train['Embarked'].mode().values[0]
X_train['Embarked'] = X_train['Embarked'].fillna(Embarked_mode_Value)

X_train['Age'] = X_train[['Pclass','Age']].apply(lambda x: (38 if x['Pclass'] == 1 else ( 30 if x['Pclass'] ==2 else 25))
                                if pd.isnull(x['Age']) else x['Age'] , axis = 1)

X_train['Family'] = X_train['SibSp'] + X_train['Parch']

X_train.drop(['SibSp','Parch'], axis=1, inplace=True)

X_train['Prefix_Name'] = X_train['Name'].str.split('.').apply(lambda x: x[0] ).str.split(',').apply(lambda y: y[1]).str.strip()

X_train.drop(['Name','Ticket'], axis=1, inplace=True)

X_train['Prefix_Name'] = X_train['Prefix_Name'].apply(lambda X : 'Rare' if X not in ['Mr','Miss','Mrs'] else X )

In [15]:
X_train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Prefix_Name
301,1,3,male,25.0,23.2500,Q,2,Mr
309,1,1,female,30.0,56.9292,C,0,Miss
516,1,2,female,34.0,10.5000,S,0,Mrs
120,0,2,male,21.0,73.5000,S,2,Mr
570,1,2,male,62.0,10.5000,S,0,Mr


In [14]:
X_train['Prefix_Name'].value_counts(normalize=True).sort_values()

Rare    0.073034
Mrs     0.141854
Miss    0.202247
Mr      0.582865
Name: Prefix_Name, dtype: float64

In [ ]:
X_train.select_dtypes(include='O').columns.tolist()

In [16]:
# One hot Encoding
for i in X_train.select_dtypes(include='O').columns.tolist():
    X_train = pd.get_dummies(X_train,columns=[i] , prefix= [i] , drop_first=True)

In [17]:
X_train.head()

,Survived,Pclass,Age,Fare,Family,Sex_male,Embarked_Q,Embarked_S,Prefix_Name_Mr,Prefix_Name_Mrs,Prefix_Name_Rare
301,1,3,25.0,23.2500,2,1,1,0,1,0,0
309,1,1,30.0,56.9292,0,0,0,0,0,0,0
516,1,2,34.0,10.5000,0,0,0,1,0,1,0
120,0,2,21.0,73.5000,2,1,0,1,1,0,0
570,1,2,62.0,10.5000,0,1,0,1,1,0,0


In [18]:
train_X = X_train.loc[: , [x for x in X_train.columns if x not in ['Survived']] ]
train_y = X_train['Survived']

In [19]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
train_X = minMaxScaler.fit_transform(train_X)
sgd_clf = SGDClassifier(random_state=3, loss='log')
sgd_clf.fit(train_X, train_y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=3, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(train_y, sgd_clf.predict(train_X))
con_mat

array([[426,  17],
       [138, 131]], dtype=int64)

In [21]:
from sklearn.metrics import precision_score, recall_score
prec_score = precision_score(train_y, sgd_clf.predict(train_X))
print("Precision Score : ", prec_score)
rec_score = recall_score(train_y, sgd_clf.predict(train_X))
print("Recall Score", rec_score)
from sklearn.metrics import f1_score
print('F1 Score', f1_score(train_y, sgd_clf.predict(train_X)))

Precision Score :  0.8851351351351351
Recall Score 0.48698884758364314
F1 Score 0.6282973621103118


In [23]:
from sklearn.metrics import classification_report
print(classification_report(train_y,sgd_clf.predict(train_X)))

             precision    recall  f1-score   support

          0       0.76      0.96      0.85       443
          1       0.89      0.49      0.63       269

avg / total       0.80      0.78      0.76       712

